In [1]:
import pandas as pd
import numpy as np

### Read data
#### During processing this file is a combination of all matched data from Kwango, Maniema, and Tshuapa
#### For this example we are using a sample of ten points



In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/ciesin-geospatial/public-summer2020/master/SNIS/sample_data.csv')
df.shape # Total number of samples

(10, 25)

In [3]:
df.head()

,NWAYPT,LATITUDE_EXCEL,LONGITUDE_EXCEL,NOM DE L'ENTITE,TYPE,OBSERVATION,TERRITOIRE,SECTEUR,GROUPEMENT,APPARTENANCE,...,GPS,COMMUNE,QUARTIER,DATETIME,ELEVATION,GEOMETRY,LATITUDE,LONGITUDE,SUBTYPE,LOCATION REVIEW PRIORITY
0,5,NaN,NaN,Poste de santé Amba Kika,PS,"Nombre d'infirmier: 1, laboratin: 0, Accoucheu...",KASONGO,MARINGA,AMBA,ETAT,...,8,NaN,NaN,2018-11-30T15:02:24Z,556.627625,POINT Z (26.528609 -4.24005 556.627625),-4.240050,26.528609,NaN,NaN
1,6,NaN,NaN,Village SENGAMALI,Village,NaN,KASONGO,MARINGA,MWINGA,NaN,...,8,NaN,NaN,2018-11-30T15:06:18Z,559.505920,POINT Z (26.541716 -4.230463 559.5059199999999),-4.230463,26.541716,NaN,NaN
2,7,NaN,NaN,Centre de santé de référence de SENGAMALI,CSR,"Nombre des médecins: 2 d'infirmier: 8, laborat...",KASONGO,MARINGA,MWINGA,ETAT,...,8,NaN,NaN,2018-11-30T15:07:40Z,559.000488,POINT Z (26.547445 -4.228114 559.000488),-4.228114,26.547445,NaN,NaN
3,8,NaN,NaN,Institut Sengamali,E.S,Nombre de garçons: 135 et de filles: 165,KASONGO,MARINGA,MWINGA,ETAT,...,8,NaN,NaN,2018-11-30T15:21:05Z,558.633789,POINT Z (26.553115 -4.223899 558.633789),-4.223899,26.553115,NaN,NaN
4,9,NaN,NaN,Fin village Sengamali,Village,NaN,KASONGO,MARINGA,MWINGA,NaN,...,8,NaN,NaN,2018-11-30T15:29:45Z,553.528503,POINT Z (26.56282 -4.228043 553.528503),-4.228043,26.562820,NaN,NaN


### Cleaning
#### Start by stripping leading and trailing spaces, and moving to all uppercase for string columns



In [4]:
def spaces_and_case(column_name, df):
    df[column_name] = df[column_name].astype(str).str.lstrip()
    df[column_name] = df[column_name].astype(str).str.rstrip()
    df[column_name] = df[column_name].apply(lambda x: x.upper())
    return df


df = spaces_and_case('TERRITOIRE', df)
df = spaces_and_case('ACCESSIBILITE', df)
df = spaces_and_case('TYPE', df)
df = spaces_and_case('APPARTENANCE', df)
df = spaces_and_case('SECTEUR', df)
df = spaces_and_case('GROUPEMENT', df)
df = spaces_and_case('AIRE DE SANTE', df)
df = spaces_and_case('ZONE DE SANTE', df)
df = spaces_and_case('NOM DE L\'ENTITE', df)
df = spaces_and_case('OBSERVATION', df)
df = spaces_and_case('COMMUNE', df)
df = spaces_and_case('QUARTIER', df)
df = spaces_and_case('NOM DU RESPONSIBLE', df)

### Spelling Variations
#### Because of the wide variety of recorded entries it is necessary to review the value counts of each column
#### These value counts populate the province reference example sheets

In [5]:
# Everything within the square brackets after .replace is replaced by the entry before the closing paren
# Example: if we are replacing vehilce and vehic with vehicule the syntax would be .replace(['vehilce', 'vehic'],'vehicule' )


# Tshuapa and Kwango
df['ACCESSIBILITE'] = df['ACCESSIBILITE'].replace(['VÉHICULE', 'VEHICUL', 'VOITURE', 'VEHUCULE', 'VEHUCUL', 'VEHUCIL','VÉHICUL',
                                                   'VEHUCE', 'AUTO', 'VIHICUL','WEHICULE'], 'VEHICULE')

df['ACCESSIBILITE'] = df['ACCESSIBILITE'].replace(['PIED', 'PIES', 'PIRDSS', 'PID', 'PIDS'], 'PIEDS')

df['ACCESSIBILITE'] = df['ACCESSIBILITE'].replace(['VEH/MOTO', 'MOTO/VEH', 'VEHICULE, MOTO', 'VÉHICULE/MOTO',
                                                   'MOTO/ VEHICULE', 'VÉHICULE / MOTO', 'MOTO/VEHICULE', 'MOTO VEHIC','VEHICULE/MOTO'], 'VEHIC/MOTO')

df['ACCESSIBILITE'] = df['ACCESSIBILITE'].replace(['M', 'MOTOS', 'M0T0','TRAVERSABLE PAR MOTO'], 'MOTO')

df['ACCESSIBILITE'] = df['ACCESSIBILITE'].replace(['H-B'], 'HORBORD')

df['ACCESSIBILITE'] = df['ACCESSIBILITE'].replace(['MOTO,PIEDS','MOTO; PIEDS', 'MOTO/PIEDS', 'MOTO PIED', 'PIED MOTO',
                                                   'MOTO  PIED', 'MOTO ET PIED', 'MOTO, ET PIEDS', 'MOTO  PIEDS', 'PIED   MOTO',
                                                  'MOTO   PIED', 'MOTO ET PIEDS', 'MOTO,ET PIEDS'], 'MOTO/PIED')

df['ACCESSIBILITE'] = df['ACCESSIBILITE'].replace([' CAMION'], 'CAMION')

df['ACCESSIBILITE'] = df['ACCESSIBILITE'].replace(['MOTO CAMINION ET PIED', 'MOTO,CAMINION, ET PIEDS', 'MOTO CAMINION PIED',
                                                  'MOTO,PIEDS ,CAMINION', 'MOTO;CAMINION,PIEDS', 'MOTO  CAMINION, ET PIEDS',
                                                  'MOTO, CAMINION  ET PIEDS ', 'PIEDS CAMINION  MOTO', 'MOTO  PIED CAMINION', 
                                                  'MOTO,COMINION ET PIEDS', 'PIED MOTO CAMINION ', 'MOTO,CAMINION ,ET PIEDS', 
                                                  'PI ET MOTO CAMINION', 'PIED MOTO CAMINION', 'MOTO; CAMINION, PIEDS',
                                                  'MOTO,CAMINION,EP', 'MOTO, CAMINION  ET PIEDS','MOTO PIED CAMINION'], 'MOTO PIED CAMION')
df['ACCESSIBILITE'] = df['ACCESSIBILITE'].replace(['PIROGUE SEULE'], 'PIROGUE')
df['ACCESSIBILITE'] = df['ACCESSIBILITE'].replace(['VEHICLE/MOTO/VELO'], 'VEHIC/MOTO/VELO')


acc_dist = df['ACCESSIBILITE'].value_counts().to_frame()
# acc_dist.head(50)
# acc_dist.to_excel('acc_dist.xlsx')

In [6]:
df['TYPE'] = df['TYPE'].replace(['REPERE GEO', 'REP GÉO', 'REER GEO', 'REPERE GEOGRAPHIQUE', 'REP G', 'REP  GEO', 'REPER GEO',
                                 'REPGEO', 'RÉP.GÉO', 'REPERE GEOGRA', 'RÉG.GÉO', 'RÉP. GÉO', 'RÉP. GÉOGRA',
                                'REF GEO','RÉPÈRE GÉO.','REP.GEO','REP, GEO', 'R.G', 'REPÈRE GÉOGRAPHI',
                                'RÉP GÉO', 'REP. GEO', 'REP,GEO', 'REP,GÉO',' REP GÉO', 'REP.GÉO',
                                'REPÈRE GÉOGRAPHIQUE','RÉP GEOGRA','REP. GÉO.', 'RÉPERE GÉO', 'REP GEQ', 'REPERE GÉOGRAPHIQUE',
                                'RIP-GÉO', 'RÉPÈRE GÉO', 'REPGEOGRAPHIQUE', 'RÉPGÉO', 'Reo.geo', 'Rep   Geo', 'Rep   Géo', 'rep  géo',
                                'REP GEO ', 'RG', 'REPERE GÉO', 'REO.GEO', 'REP   GEO', 'REP   GÉO', 'REP  GÉO', 'REP. GEO HHV',
                                'REP.G', 'REPERE GÉO', 'RÈPERE GÉO','GÉO','RÉ. GÉOGR','RÉG.GÉOG','RÉG.GÉOGR','REP','REP  GEO.',
                                'REP  GEOGR','RÉP  GEOGR','REP GEO.','REP GÉO.','RÉP GEOGR','REP GEOGRAPH','REP- GEO',
                                'REP-GEO','REP, GÉO','RÉP, GÉO','RÉP,GÉO','REP.','REP. GÉO','RÉP. GEO','REP. GEO.','RÉP. GÉOGR',
                                'REP.GEOG','RÉP.GÉOGE','REP.GEOGR','RÉP.GÉOGR','REP.GEOR','RIP','RIP GEO','RIP.GÉO','GEO','GEP GEO'], 'REP GEO')




df['TYPE'] = df['TYPE'].replace(['REPERE'], 'REP')
df['TYPE'] = df['TYPE'].replace(['VILL', 'VIL', 'VII', '¨VILL', 'VILAGE','VALLAGE','VILLA','VILLGE',
                                'VLLAGE','-   VILLAGE','VILLAGE/MISSION'], 'VILLAGE')

df['TYPE'] = df['TYPE'].replace(['P.S', 'P S','POSTE'], 'PS')
df['TYPE'] = df['TYPE'].replace(['HAM','HAMON','HAMMEAU','HAMEA','HOM','HANEAU', 'HAMAU', 'HAMAUX', 'HAMO',
                                '- HAMEAU','HAMEAUX'], 'HAMEAU')
df['TYPE'] = df['TYPE'].replace(['EP.','E.P', 'E P','E P','E  P','E,P','Ë.P','RP','EP BENENGE','EPS'], 'EP')
df['TYPE'] = df['TYPE'].replace(['SCC'], 'SSC')
df['TYPE'] = df['TYPE'].replace(['I.P', 'I P','I¨P','-   IP','PI'], 'IP')
df['TYPE'] = df['TYPE'].replace(['AV','A.V','AV.'], 'AVENUE')
df['TYPE'] = df['TYPE'].replace(['RIVIERE','RIV','RIVIERE TSHUAP','RIVIRRE'], 'RIVIÈRE')
df['TYPE'] = df['TYPE'].replace(['DEBUT DE L\' AVENUE'], 'DEBUT DE L\'AVENUE')
df['TYPE'] = df['TYPE'].replace(['ADM.','ADMST','ADMIN','ADMIN/POINTINT','ADMN'], 'ADM')
df['TYPE'] = df['TYPE'].replace(['LYCEE'], 'LYCÉE')
df['TYPE'] = df['TYPE'].replace(['EMAT', 'E MATER', 'E.M','E MAT.','E MAT','EC.M'], 'EM')
df['TYPE'] = df['TYPE'].replace(['C S', 'C.S','C .S','CENTRE DE SANTE BENDJALI'], 'CS')
df['TYPE'] = df['TYPE'].replace(['INST.','INSTITUT', 'INSITUT', 'INSTIT', 'INST,','ISNT','INST;','INST;.','INT','IST','VILLAGE INST'], 'INST')
df['TYPE'] = df['TYPE'].replace(['LMT.', 'LMT', 'L IMITE', 'LIM', 'LÎMITE','LIMITES','LIMITA','LJMITE','LIMITE GEO'], 'LIMITE')
df['TYPE'] = df['TYPE'].replace(['FORRET.','FORRET'], 'FORET')
df['TYPE'] = df['TYPE'].replace(['DISP.'], 'DISP')
df['TYPE'] = df['TYPE'].replace(['S.S.C','SC'], 'SSC')
df['TYPE'] = df['TYPE'].replace(['AMON'], 'AMO')
df['TYPE'] = df['TYPE'].replace(['UNIV.','UNIV'], 'UNIVERSITÉ')
df['TYPE'] = df['TYPE'].replace(['AEDROME','AERO'], 'AERODROME')
df['TYPE'] = df['TYPE'].replace(['BLV','BLVRD'], 'BLVD')
df['TYPE'] = df['TYPE'].replace(['C.S.R'], 'CSR')
df['TYPE'] = df['TYPE'].replace(['CAMPEMENT'], 'CAMP')
df['TYPE'] = df['TYPE'].replace(['RIERE','REVIERE'], 'RIVIÈRE')
df['TYPE'] = df['TYPE'].replace(['E.S'], 'ES')
df['TYPE'] = df['TYPE'].replace(['LYC'], 'LYCÉE')
df['TYPE'] = df['TYPE'].replace(['BR.ADM','BURADM'], 'BUREAU')
df['TYPE'] = df['TYPE'].replace(['C HOSP','C.HOSP'],  'CH')
df['TYPE'] = df['TYPE'].replace(['H.GR','HGR-WM'], 'HGR')
df['TYPE'] = df['TYPE'].replace(['MARHE'], 'MARCHE')
df['TYPE'] = df['TYPE'].replace(['EGISE','EG'], 'EGLISE')



type_dist = df['TYPE'].value_counts().to_frame()

In [7]:
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['L\'ÉTAT', 'L’ETAT','L\'ETAT', 'L\'état','ÉTAT','ETATIQUE','-(L\'ÉTAT)','(L\'ÉTAT)','ETAAT','ET','ETA','EYAT','ETTAT','ETET',
                                                'L\'ETA T','L"ETAT','EETAT','EAT','NC(L\'ETAT','NC( L\'ETAT)','ETAT (CLC)','NC','L\'ETAT(NC)','NC(L\'ETAT)','NC L\'ETAT','OFFICIEL/ETAT',
                                                'NC (L\'ETAT)','L\'ETA T(NC)'], 'ETAT')

df['APPARTENANCE'] = df['APPARTENANCE'].replace(['CONV PROTESTANTE','CONVENTIONNEE PROTESTANTE 25È CEK','CONVENTIONNE PROTESTANT','CONV.PROTESTANTE','CONVENTIONNÉE PROTESTANTE',
                                                'CONV PROTESTANTE CEFMC','CONV PROTESTENTE','CONV PROTEST', 'CONV  PROTESTANTE', 'CONV PR', 'CONVENTIONNE PR','CONVENTIONNE PROT',
                                                'CONVENTIONNÉ PROTESTENTE','CONVENTIONNÉ PROTSTANTE','CONV PROT','CONV.PROT.CEK', 'CONVENTIONNE PROTE', 'EGIE PROTETANT',
                                                'CONVETIONNÉ PROTESTANTE','CONVENTIONNÉ PROTESTANTE','CONV.PROT. CEK', 'CONFETINNEL PROTE', 'ECV.PROTESTANTE', 'PROTEST CEFMC',
                                                'CONVENTIONNE PROTESTENTE','CONV.PROTESTENT','CONV.PROTESTENTE CEK','CONV. PROTESTENT', 'ECV.PROTESTANT','POTESTANT CMCO',
                                                'CONVENTIONNE  PROTESTENT','CONVENTIONÉ PROTEST','CONVENTIONNE  PROTESTANT','CONV. PROTEST','CONV PROTESTENT', 'PROTESTENT/CEFMC',
                                                'CONVENTIONNÉ PROTETANTE','CONVENTIONNEE PROTESTANTE CEBAWB','CONV PROSTANTE','CONVENTIONNELLE PROTESTANTE', 'PROTESTANT CMCO',
                                                'CONV  PROSTANTE','CONFESSIONNELLE PROTESTANTEΜ','CONV. PROTESTANTE','CONV. PROTESTANT','CONV.PROTESTANT', 'PROTESTANT CEK',
                                                'CONV.PROT. CADC','CONV. PROTESTENT/ESECO','CONV.PROTEST','CONVENTINNÉ PROTESTANTE','CONVENSIONÉE PROTESTANTE', 'PROTESTATNTE',
                                                'CONVENTIONNEE PROTESTANTE 44ÈCBFA','COVENTIONNÉ PROTESTENTE','CONV. PROT','CONV.PROTESTENT/CEMCO','¨CONVENTIONNE PROTESTANT',
                                                'CONVENTIONNEE PROTESTANTE 53È CBK','CONVENTIONNE  PRETESTENTE','CONVENTIONÉ PROTESTANTE','CONVENTIONNE ¨PROTESTANT',
                                                'CONV.PROT. CBSK','CONVENTIONNEE PROTESTANTE','CONV PROTESTENT/ESECO','CONVETIONNE PROTESTANTPIED','CONV.PROTEST/CLC', 'PROTESTENT/CEMCO',
                                                'CNVENTIONNE PROTESTANT','COVENTIONNÉ PROSTESTENTE, ECOLE EN DURABLE MAIS EN MAUVAIS ETAT  PAS DE PIPITRE','CONV.PROTEST/ESECO',
                                                'CONV. PROTESTENT/CELC','CONV PROTESTENT/CELC','CONVENTIONNEE PROTESTANTE CFO','CONVENT PROTESTANTE', 'CONF. PROT.','CONF. PROT',
                                                'PROTESTANTE','PROTESTENT','PR0TESTANTE','PRORESTENT','PR0TESTANT','PROTEST','CONV PROTESTANT','CONFESSION PROTESTANTE', 'PROTESTENT CEMCO',
                                                'CON. PROTESTANT','CONFESSION   PROTESTANTE','CONFESSION  PROTESTANTE','CONV.PROT','CP','ECOLE PROTESTANTE','ECP',
                                                'REVEIL CONVEVENTIONNE PROTESTANTE'], 'PROTESTANT')

df['APPARTENANCE'] = df['APPARTENANCE'].replace(['CONVENTIONNE CATHOLIQUE','CONV.CATHOLIQUE','CONVENTIONNÉE CATHOLIQUE', 'CONFETINNELLE CATH', 'CONV CATH', 'CONV CATHOLIQ',
                                                'CONVENTIONNEE CATHOLIQUE','CONVENTIONNÉ CATHOLIQUE ','CONVENTIONNÉ CATHOLIQUE','CONV CATHOL', 'CONV CATOLIQUE', 'CTHOLIQUE',
                                                'CONV. CATHOLIQUE','CONVENTIONNE  CATHOLIQUE','CONVENTIONNÉ  CATHOLIQUE','CONVENTIONNÉE  CATHOLIQUE','CONVENTIONNNE CATHOLIQUE',
                                                'CONVENTION CATHOLIQUE','CONV.CATH','COVENTIONE CATHOLIQUE','CONV COTHOLIQUE','CONV,CATHOLIQUE','CONVENTIONÉ CATHO','CONV;CATHOLIQUE',
                                                'ETAT  CONVENTIONE CATHOLIQUE','CONVETIONNE CATHOIUE','CONVCATHOLIQUE','CONV, CATHOLIQUE','CONVENTIONNELLE CATHOLIQUE',
                                                'CATH','CATHOIQUE','CATHOLOQUE','CATHOILQUE','. CATHOLIQUE', 'CONV CATHOLIQUE', 'CON CATHOLIQUE', 'CONVENTIONNÉ CATHOLIQ',
                                                'CONF. CATH.','CONFESSIONNELLE CATHOLIQUE','CONCESSION CATHOLIQUE','CONCESSION CATH','CONF.  CATH.', 'EC CATHOLIQUE',
                                                'CONF.CATH.','CONFE CATH','CONFESSIN CATHOLIQUE','CONFESSIO  CATHOLIUE','CONFESSION   CATHOLIQUE', 'CONFESSION CATHOLIQUE',
                                                'CONFESSION  CATHOLIQUE','CONVE. CATHOLIQUE','CONVENT CATHOLIQUE','CONVENTIONNE CATH','CONVENTIONNE CATHO','CONVENTIONNE CATHOL',
                                                'ECCAT','ECCATH','ECCATHOL','ECC','ECCO','ECC0','ECECO','ECV. CATHOLIQUE','ECV.CATHOLIQUE','EGLISE CATHOLIQUE','EMC CATHOLIQUE'], 'CATHOLIQUE')


df['APPARTENANCE'] = df['APPARTENANCE'].replace(['SAKIMA MINIERE','SOCIETE SAKIMA'], 'SAKIMA')


df['APPARTENANCE'] = df['APPARTENANCE'].replace(['OFFICIELLE'], 'OFFICIEL')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['REVEIL','RÉVEIL','RÉVEILLE'], 'REVEILLE')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['CONV.'], 'CONVENTION')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['NON CONVENTIONNE','NON COVENTIONNEE','NON CONV'], 'NON CONVENTIONNEE')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['CONV.METHODISTE','CONV METHOD','CONFETINNEL METODISTE','CONV METHODISTE UNIE','CONVENT METHODISTE','CONVENTIONNEE 28E METHODISTE',
                                                'CONVETIONNEE 28E METHODISTE','CONVMETHODISTE','ECM','ECV.METHODISTE','CONV METHODISTE'],'METHODISTE' )
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['PRIVÉ','¨PRIVEE','PRIVEE','PRIVÉE','PRIVÉ  INTEGRÉ','PRIVE AGREE','PRIVÉ/OSD','PRIVEE  AGREE','PRIVEE AGREE','PRIVVE'], 'PRIVE')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['BRAHNAM','BRANAM','BRANHAM'], 'BRANAHAM')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['19E  CEEBCO','19E CEBCO'], '19E CEEBCO')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['BRAHAMANISTE','BRANAMISTE'], 'BRANAHAM')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['CONFESSION'], 'CONFESSIONNELLE')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['5E  COMMUNAUTE CELPA','5E COMM . CELPA','5E COMMUN. CELPA','5E FEPACO ZAMBE MALAMU','CONV 5È CELPA',
                                                '5ÉM CELPA','5ÈME CELPA','5È CELPA','CONV. 5È CELPA'], '5E CELPA')

df['APPARTENANCE'] = df['APPARTENANCE'].replace(['7E JOUR ADVENTISTE','7ÉM ECC','7ÉM ECGCO'], '7E CEGC')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['81E C. METHODISTE','81E CM','CONV. 81È COM METHODISTE UNIE'], '81E COMMUN METHODISTE')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['EGLISE DE REVEIL','EGLISE DE REVIEL','EGLISE REVEIL'], 'EGLISE DE REVEILLE')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['MUSILMAN','MUSULMANE','MUSULMANS','CONV MUSULMANE'], 'MUSULMAN')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['NEO','NEO AOSTOLIQUE','NEO AP','NEO APOST','NEO APOSTOL','NEO APPOSTOLIQUE','NEO-APOSTOLIQUE','NEOAPOSTOLIQUE'], 'NEO APOSTOLIQUE')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['T  J','T.JEHOVA','TEMOIN','TEMOIN DE JEHOVA','TEMOIN DE JEHOVAH','TEMOIN DE JOHOVAH','TEMOIN DE JOVAH',
                                                 'TEMOINS DE JEHOV','TEMOINS DE JEHOVA','TEMOINS DE JÉHOVA','TEMOINS DE JEHOVAH','TÉMOINS DE JEHOVAH','TEMOINS DE JOVAH','JEHOVISTE'], 'TÉMOINS DE JÉHOVAH')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['8È CEPAC','8ECEPAC','CONV 8E CEPAC','CONV 8E CEPAC CONFESSIONNELLE','CONV.  8ÈME CEPAL'], '8E CEPAC')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['CON. ISLAMIQUE','CONFESSION   ISLAMIQUE','CONFESSION  ISLAMIQUE','CONFESSION ISLAMIQUE','CONV ISLAM/','CONV ISLAMIQUE',
                                                'CONV. ISLAMIQUE','CONV.ISLAMIQUE','CONVENT ISLAMIQUE','CONVENTIONNE ISLAMIQUE','CONVENTIONNEISLAMIQ','ECV.ISLAMIQUE','ISLAM','IBLAMIQUE'], 'ISLAMIQUE')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['CONVENTIONNEE', 'CONVENTIONNÉE'], 'CONVENTIONNELLE')
df['APPARTENANCE'] = df['APPARTENANCE'].replace(['KIMBAQUISTE','KIMBANGWISTE','KIMBANGUSTE','KIMBANGUST',
                                                'CONFESSION   KIMBANGISTE','CONFESSION  KIMBANGISTE','CONFESSION KIMBANGISTE','CONFESSIONNELLE KIMBANGUISTE','CONV  KIMBANGUISTE',
                                                 'CONV KIMB','CONV KIMBAGIUSTE','CONV KIMBAGUISTE','CONV KIMBAGUITE','CONV KIMBANGUISTE','CONV KIMBNGUISTE','CONV. KIMBANGISTE','CONV. KIMBANGUISTE',
                                                 'CONV.KIMBANGUIST','CONV.KIMBANGUISTE','CONVENT KIMBANGUISTE','CONVENTION  KIMBANGUISTE','CONVENTIONNEE KIMBANGUISTE','CONVENTIONNÉE KIMBANGUISTE',
                                                 'EC KIMBAGUITE','EC KIMBANGUISME','EKIMBANGUISTE','KIMBAGUISTE','KIMBANGISTE','KIMBANGUISME'], 'KIMBANGUISTE')


app_dist = df['APPARTENANCE'].value_counts().to_frame()

In [8]:
df['ZONE DE SANTE'] = df['ZONE DE SANTE'].replace(['BONDE'], 'BOENDE')
df['ZONE DE SANTE'] = df['ZONE DE SANTE'].replace(['BUSNGA'], 'BUSANGA')
df['ZONE DE SANTE'] = df['ZONE DE SANTE'].replace(['MWELA -LEMBWA', 'MWELA - LEMBWA','MWELA-LEMBWA'], 'MWELA LEMBWA' )
df['ZONE DE SANTE'] = df['ZONE DE SANTE'].replace(['WAMBA LUADI','WAMBA-LUADI'], 'WAMBA LWADI')
df['ZONE DE SANTE'] = df['ZONE DE SANTE'].replace(['KASONGO-LUNDA'], 'KASONGO LUNDA')
df['ZONE DE SANTE'] = df['ZONE DE SANTE'].replace(['LUBTU'], 'LUBUTU')
df['ZONE DE SANTE'] = df['ZONE DE SANTE'].replace(['KIMBAU'], 'KIMBAO')
df['ZONE DE SANTE'] = df['ZONE DE SANTE'].replace(['RURALE DE KAJIJI'], 'KAJIJI')
df['ZONE DE SANTE'] = df['ZONE DE SANTE'].replace(['RURALE DE KAHEMBA'], 'KAHEMBA')


zone_dist = df['ZONE DE SANTE'].value_counts().to_frame()

In [9]:
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['YANYANGOMBULA'], 'YANYANGO MBULA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['AMANKOYO'], 'AMANKOYOLA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['AS BATANGA'], 'BATANGA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['AS BOENDE 2 KIMBANGU'], 'BOENDE 2 KIMBANGU')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['AS BOLEMBA'], 'BOLEMBA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['AS BONGOY'], 'BONGOY')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['AS CDCC'], 'CDCC')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['AS ILOMBE'], 'ILOMBE')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['AS LIFOKU'], 'LIFOKU')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['AS LIFOKU'], 'LIFOKU')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['AS MOREIE LOUISE'], 'MARIE LOUISE')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['AS NDJAFA'], 'NDJAFA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['AS NSONGE'], 'NSONGE')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['BEFORY'], 'BEFORI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['BOENDE II NSELE'], 'BOENDE 2 NSELE')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['BOKONDJI'], 'BEKOMI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['DJOLU MUKE'], 'DJOLU MOKE')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['DURI PANGI'], 'DURI MPANGI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['E. NSAMBA'], 'EANDJA NSAMBA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['EKUKOLA'], 'EKUKOLA BOLONGO')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['ESPRIT'], 'SAINT ESPRIT')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['FESHI 1','FESHI 2'], 'FESHI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['FIMBONYINGI'], 'FIMBO NYINGI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['IKENGOLAKA II'], 'IKENGOLAKA 2')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['IMBO NSAMBA'], 'IMBO')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KABAMBANGI'], 'KABAKA MBANGI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KABEYAMBAMBA'], 'KABEYA MBAMBA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KALUNGAMUGABO'], 'KALUNGA MUGABO')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KAMA II'], 'KAMA 2')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KAMBAKADIMA'], 'KADIMA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KAMBANGUYA'], 'KAMBA NGUYA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KAMBASENGO'], 'KAMBANSENGO')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KAPAY-LONO'], 'KAPAY LONO')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KAPINI PINI'], 'KAPINIPINI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KEMBEYULE'], 'KEMBE YULE')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KENGE II'], 'KENGE 2')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KENNGE MUNIUNGU'], 'KENGE MUNIUNGU')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KHUNGANENE'], 'KUNGANENE')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KIALAKAMBA'], 'KIALA KAMBA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KIKONGOLWASA'], 'KIKONGO LWASA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KIMBASEKE I'], 'KIMBASEKE 1')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KOLOMBONYAMA'], 'KALOMBENYAMA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['LABUNDA'], 'KABUNDA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['LIKUKU II'], 'LIKUKU')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['LIYANGOLA II'], 'LIANGOLA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['LOKATA'], 'BOKELA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['M\'PIALA'], 'MPIALA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['MAZEMBA'], 'MAZEMBE')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['MBINDATSEKE'], 'MBINDANSEKE')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['MC MONDOMBE'], 'MONDOMBE')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['MPUTSUANGA'], 'MPUTSHUANGA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['MUSANGUTSAY'], 'MUSANGU TSAY')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['MUTAY'], 'MUTAYI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['MWANA-UTA'], 'MWANA UTA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['MWANAMUYOMBO'], 'MUANA MUYOMBO')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['MWELA  TSAY'], 'MUSANGU TSAY')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['MWELA KASA'], 'MWELA KASSA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['MWELA MBWANDU'], 'MUELA MBWANDU')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['N\'TEMO'], 'NTEMO')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['NDANDALUKALA'], 'NDANDA LUKALA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['NDEOMANONO'], 'NDEO MANONO')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['NGOMBE-NTUMBA'], 'NGOMBETUMBA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['NGUDI NKAMA'], 'NGUDIKAMA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['NGUNDU-MAYALA'], 'NGUNDU MAYALA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['NZAMBA FWANGI'], 'NZAMBA FUANGI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['NZASI -MWADI'], 'NZASIMWADI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['PANZI/MAKITA'], 'PANZI/ MAKITA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['PIANIA I'], 'PUNIA 1')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['PILOTE WEMA'], 'WEMA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['PUNIA II'], 'PUNIA 2')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['SALAMABILA ET CAMP- CENTRAL'], 'CAMP CENTRAL')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['SALUMU'], 'SALUMU BASI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['SHATUNGISHI'], 'SHATUNGUSHI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['SOUS-MARIN'], 'SOUS MARIN')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['ST AUGUSTIN'], 'SAINT AUGUSTIN')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['SUITE MANZENGELE'], 'MANZENGELE')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['SUKA-MBUNDU'], 'SUKA MBUNDU')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['SWA  BANGU'], 'SWA BANGU')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['SWA-YAMVU'], 'KITENDA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['SWA/KIBULA'], 'SWAKIBULA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['TAMBUTSEKE'], 'TAMBU TSEKE')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['TEMBO-KUNTUALA'], 'TEMBO KINTWALA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['TSAKALA  NKUKU'], 'TSAKALA KUKU')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['TSAKALA MBEWA'], 'TSAKALA MBEWU')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['TSHAKALA PANZI'], 'TSAKALA PANZI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['WAMBA-LUADI'], 'WAMBA LUADI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['YAIKALA'], 'YALIKALA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['YAMVU-KILUNGA'], 'YAMFUKILUNGA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['YANSEKO'], 'YASEKO')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['YANYANGO MBULA'], 'YANYANGOMBULA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['YOLONGA III'], 'YOLONGA 3')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['YOLONGA1'], 'YOLONGA 1')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['ZHINABUKETE'], 'ZHINABUNKETE')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['CITE POPO 1','CITE POPO 2'], 'CITÉ POPOKABAKA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KIBUNDA'], 'KIBUNDA 1')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KAMBWELA'], 'KAMBUELA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['NGONDIMUTOMBO'], 'BANGONGO')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KIKUTI','KIKWATI'], 'BAMBA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['SEFU/SWA-TENDA'], 'KITENDA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KITANGU'], 'KIMWAMBU')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['MALAMBU'], 'MUYALALA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['SWA-ILUNGA'], 'MWANA UTA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['SAINT  PIERRE'], 'KOBO')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['MVUFU'], 'YENGA LUSUNDJI')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['KIBENGA'], 'MAWANGU')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['YOLONGA 1'], 'YOLONGA')
df['AIRE DE SANTE'] = df['AIRE DE SANTE'].replace(['LOKOMA'], 'NKEMBE')

aire_dist = df['AIRE DE SANTE'].value_counts().to_frame()

In [10]:
df['TERRITOIRE'] = df['TERRITOIRE'].replace(['KASONGO LUNDA', 'KASONG-LUNDA'], 'KASONGO-LUNDA')
df['TERRITOIRE'] = df['TERRITOIRE'].replace(['K°-LUNDA'], 'K°LUNDA')
df['TERRITOIRE'] = df['TERRITOIRE'].replace(['BONDE'], 'BOENDE')
df['TERRITOIRE'] = df['TERRITOIRE'].replace(['KABAMBARE BB SALAMABILA','KABAMBARE BB SALAMABILAMU','KABAMBAREBB AWAMAZ','KABAMBAREBB AWAMAZKANYENGELEETA','KABAMBAREBB SAL','KABAMBAREBB WAMAZA','KABAMMBARE','KAMBARE'], 'KABAMBARE')
df['TERRITOIRE'] = df['TERRITOIRE'].replace(['KENGEII'], 'KENGE II')
df['TERRITOIRE'] = df['TERRITOIRE'].replace(['LUBTUTU','LBUTU'], 'LUBUTU')
df['TERRITOIRE'] = df['TERRITOIRE'].replace(['MONOTO'], 'MONKOTO')
df['TERRITOIRE'] = df['TERRITOIRE'].replace(['PUNIK'], 'PUNIA')


terr_dist = df['TERRITOIRE'].value_counts().to_frame()

In [11]:
df['SECTEUR'] = df['SECTEUR'].replace(['KASONG-LUNDA', 'KASONGO-LUNDA' ], 'KASONGO LUNDA')
df['SECTEUR'] = df['SECTEUR'].replace(['PELENDE   NORD', 'PELENDE  NORD', 'PELENDENORD'], 'PELENDE NORD')
df['SECTEUR'] = df['SECTEUR'].replace(['SWA- TENDA', 'SWA-TENDA'], 'SWA/TENDA')
df['SECTEUR'] = df['SECTEUR'].replace(['GANA KETI'], 'GANAKETI')
df['SECTEUR'] = df['SECTEUR'].replace(['AMWE'], 'AMBWE')
df['SECTEUR'] = df['SECTEUR'].replace(['B. B LULINDI','B.B  LULINDI','B.B. LULINDI','B.BLULINDI','BB LULINDI'], 'B.B LULINDI')
df['SECTEUR'] = df['SECTEUR'].replace(['BABANE'], 'BABENE')
df['SECTEUR'] = df['SECTEUR'].replace(['BENY -SAMBA','BENYE   SAMBA','BENYE  SAMBA','BENYE-SAMBA'], 'BENYE SAMBA')
df['SECTEUR'] = df['SECTEUR'].replace(['LOANDAO'], 'LOANDO')
df['SECTEUR'] = df['SECTEUR'].replace(['MAMBA  KASENGA','MAMBA-KASENGA'], 'MAMBA KASENGA')
df['SECTEUR'] = df['SECTEUR'].replace(['WAKABONGO 2'], 'WAKABANGO 2')


sec_dist = df['SECTEUR'].value_counts().to_frame()

In [12]:
df['GROUPEMENT'] = df['GROUPEMENT'].replace(['BÎTADI'], 'BITADI')
df['GROUPEMENT'] = df['GROUPEMENT'].replace(['TSAKUNKAKABATU'], 'TSAKUNKAKA BATU')
df['GROUPEMENT'] = df['GROUPEMENT'].replace(['MWANA UTA'], 'MWANA-UTA')
df['GROUPEMENT'] = df['GROUPEMENT'].replace(['MUSANZINGOMBO'], 'MUSANZI NGOMBO')
df['GROUPEMENT'] = df['GROUPEMENT'].replace(['MULOPONDINDI'], 'MULOPO NDINDI')
df['GROUPEMENT'] = df['GROUPEMENT'].replace(['PENETATAMBA'], 'PENEKATAMBA')
df['GROUPEMENT'] = df['GROUPEMENT'].replace(['PENE  MWIMBA','PENE -MWIMBA'], 'PENE-MWIMBA')
df['GROUPEMENT'] = df['GROUPEMENT'].replace(['NGOMBENYAMA'], 'NGOMBEMUNA')
df['GROUPEMENT'] = df['GROUPEMENT'].replace(['MUHIYA2'], 'MUHIYA 2')
df['GROUPEMENT'] = df['GROUPEMENT'].replace(['MUHIYA I', 'MUHIYA1'], 'MUHIYA 1')
df['GROUPEMENT'] = df['GROUPEMENT'].replace(['MUDIKIKAMBA'], 'MUDKIKAMBA')
df['GROUPEMENT'] = df['GROUPEMENT'].replace(['KASONGO-LUNDA'], 'KASONGO LUNDA')

group_dist = df['GROUPEMENT'].value_counts().to_frame()

In [13]:
df['COMMUNE'] = df['COMMUNE'].replace(['PELENDDE NORD','PELENDE NODR', 'PELNDE NORD', 'PELEND NORD', 'PELEND NORD',
                                       'PELENDE ONRD','PELENDE  NORD','PELEND  NORD','PLENDE NORD','PELNDE  NORD','PELENDE  NODR' ], 'PELENDE NORD')
df['COMMUNE'] = df['COMMUNE'].replace(['COMMUNE DEPANGI','COMMUNEDE PANGI'],'COMMUNE DE PANGI')

com_dist = df['COMMUNE'].value_counts().to_frame()

In [14]:
df['QUARTIER'] = df['QUARTIER'].replace(['SWA  KASONGO'], 'SWA KASONGO')
df['QUARTIER'] = df['QUARTIER'].replace(['SWA  YAMFU'], 'SWA YAMFU')
df['QUARTIER'] = df['QUARTIER'].replace(['MIRIPA'], 'MIRIPIA')
df['QUARTIER'] = df['QUARTIER'].replace(['MUFALA  II'], 'MUFALA II')
df['QUARTIER'] = df['QUARTIER'].replace(['NINGOBESA'], 'NIONGOBESSA')


quarter_dist = df['QUARTIER'].value_counts().to_frame()

In [15]:
df['NOM DU RESPONSIBLE'] = df['NOM DU RESPONSIBLE'].replace(['KAZADI  AKIM'], 'KAZADI AKIM')
df['NOM DU RESPONSIBLE'] = df['NOM DU RESPONSIBLE'].replace(['ANDRE SHAKO LOLONGA','ANDRÉ SHAKO LOLONGA'], 'ANDRÉ SHAKO  LOLONGA')
df['NOM DU RESPONSIBLE'] = df['NOM DU RESPONSIBLE'].replace(['IDUMBA  WITAMWINO'], 'IDUMBA WITAMWINO')
df['NOM DU RESPONSIBLE'] = df['NOM DU RESPONSIBLE'].replace(['KABALA NGONGO','KABALA NGONGO ASER'], 'KABALA NGONGO ARSENE')
df['NOM DU RESPONSIBLE'] = df['NOM DU RESPONSIBLE'].replace(['KAKULI AWAZI'], 'KAKUKI AWAZI')
df['NOM DU RESPONSIBLE'] = df['NOM DU RESPONSIBLE'].replace(['KIKUNI NGUMBI','KIKUNI INGIMBI','KIKUNI  NGIMBI'], 'KIKUNI NGIMBI')
df['NOM DU RESPONSIBLE'] = df['NOM DU RESPONSIBLE'].replace(['LIKUTA SABITI LEONARD'], 'LIKUTA SABITI LÉONARD')


name_dist = df['NOM DU RESPONSIBLE'].value_counts().to_frame()

### Join with Clean Zone List
#### Joining with an external list that contains unique identifies for health zones, health areas, and provinces. 
#### This zone list file is not available in this repository, so the next cell will not run. 
#### See the following cell for the final list of columns

In [16]:
clean_zone_df = pd.read_excel('/Users/peycke/Documents/proj/grid3/3_province_guide/clean_zone_upper.xlsx')
joint_df = pd.merge(df, clean_zone_df, on=['ZONE DE SANTE'])
joint_df = joint_df.drop(columns=['DPS_ID', 'DPS_UID','DATE']) # Drop empty, unwanted, or duplicate columns

In [17]:
joint_df.columns

Index(['NWAYPT', 'LATITUDE_EXCEL', 'LONGITUDE_EXCEL', 'NOM DE L'ENTITE',
       'TYPE', 'OBSERVATION', 'TERRITOIRE', 'SECTEUR', 'GROUPEMENT',
       'APPARTENANCE', 'ACCESSIBILITE', 'NOM DU RESPONSIBLE', 'AIRE DE SANTE',
       'ZONE DE SANTE', 'GPS', 'COMMUNE', 'QUARTIER', 'DATETIME', 'ELEVATION',
       'GEOMETRY', 'LATITUDE', 'LONGITUDE', 'SUBTYPE',
       'LOCATION REVIEW PRIORITY', 'ZS_UID', 'ZS_PROVINCE', 'PROVINCE_CODE',
       'PROVINCE_NAME'],
      dtype='object')

### Save Processed Versions

In [18]:
# Check types before saving
# Lat Lon must be numeric to facilitate mapping
cols_list = joint_df.dtypes.to_frame()
# cols_list

In [19]:
# Save full data set to excel
joint_df.to_excel('3_province_cleanedV1.xlsx')

### Create Reference Sheets
#### These reference sheets contain information on variable frequency

In [20]:
def full_reference_sheet():
    writer = pd.ExcelWriter('full_reference_sheet.xlsx', engine='xlsxwriter')
    # Write each dataframe to a different worksheet.
    acc_dist.to_excel(writer, sheet_name='ACCESSIBILITE')
    app_dist.to_excel(writer, sheet_name='APPARTENANCE')
    type_dist.to_excel(writer, sheet_name='TYPE')
    zone_dist.to_excel(writer, sheet_name='ZONE DE SANTE')
    aire_dist.to_excel(writer, sheet_name='AIRE DE SANTE')
    terr_dist.to_excel(writer, sheet_name='TERRITOIRE')
    sec_dist.to_excel(writer, sheet_name='SECTEUR')
    group_dist.to_excel(writer, sheet_name='GROUPEMENT')
    com_dist.to_excel(writer, sheet_name='COMMUNE')
    quarter_dist.to_excel(writer, sheet_name='QUARTIER')
    name_dist.to_excel(writer, sheet_name='NOM DU RESPONSIBLE')
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

def location_reference_sheet(province_name):
    writer = pd.ExcelWriter(f'{province_name}_location_reference_sheet.xlsx', engine='xlsxwriter')
    # Write each dataframe to a different worksheet.
    zone_dist.to_excel(writer, sheet_name='ZONE DE SANTE')
    aire_dist.to_excel(writer, sheet_name='AIRE DE SANTE')
    terr_dist.to_excel(writer, sheet_name='TERRITOIRE')
    sec_dist.to_excel(writer, sheet_name='SECTEUR')
    group_dist.to_excel(writer, sheet_name='GROUPEMENT')
    com_dist.to_excel(writer, sheet_name='COMMUNE')
    quarter_dist.to_excel(writer, sheet_name='QUARTIER')
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    
def location_independent_reference_sheet():
    writer = pd.ExcelWriter('location_independent_reference_sheet.xlsx', engine='xlsxwriter')
    # Write each dataframe to a different worksheet.
    acc_dist.to_excel(writer, sheet_name='ACCESSIBILITE')
    app_dist.to_excel(writer, sheet_name='APPARTENANCE')
    type_dist.to_excel(writer, sheet_name='TYPE')
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    
    
    

In [21]:
full_reference_sheet()
# location_reference_sheet('tshuapa') # Substitue province name here if associated data is provided
location_independent_reference_sheet()